# Regresja Liniowa
  1. Wczytanie danych
  2. Wizualizacja danych
  3. Regresja jednowymiarowa
  7. Implementacja algorytmu GradientDescent
  5. Closed-form solution
  6. Porównanie z Tensorflow

## Import bibliotek
Zaimportuj niezbędne biblioteki, takie jak *numpy, pandas, files* z *google.colab* oraz *io*

## Wczytaj plik z danymi do DataFrame
Plik, który należy załadować znajdue się tutaj [House sales train](https://szymon.bobek.re/kc_house_data_train.csv)

**Uwaga** Jest to inny plik, niż ten używany podczas ostatnich zajęć.

## Wyświetl scatterplot ceny od sqtf_living

## Wyświetl scatterplot ceny od bathrooms

## Regresja jednowymiarowa
### Podział zbioru danych
Podziel zbiór danych na dwie części w proporcjach 66% na zbiór uczacy i 34% na zbiór testowy.
Wykorzystaj [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
from sklearn.model_selection import train_test_split



### Wersja scikit-learn

In [ ]:
from sklearn import linear_model
import matplotlib.pyplot as plt


# Poniżej dwie wersje algorytmu, przetestuj obie i na końcu porównaj wyniki
# Zobacz, że w pierwszej wersji nie podajemy praktycznie żadnych parametrów

learning_rate = 0.01

#regr = linear_model.LinearRegression()
#regr = linear_model.SGDRegressor(learning_rate='constant', eta0=learning_rate)

# Naucz model wykorzystując zbiór treningowy
regr.fit(Tr['bathrooms'].values.reshape(-1,1), Tr['price'])

theta_1 = regr.coef_[0]
theta_0 = regr.intercept_

# Poniże kod do wizualizacji. Przerrób go na funkcję,
# tak aby można go było wykorzystać w dalszych ćwiczeniach
ax=Tr.plot.scatter(x='bathrooms',y='price')
x = np.linspace(0,14,100)
y = theta_1*x+theta_0
ax.plot(x,y,color='green')

#### Wykorzystaj funkcję [predict](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.predict) na zbiorze testowym, żeby wyznaczyć ceny szacowane przez model.

#### Wykorzystj funkcję [mean_squared_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) oraz [r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) żeby oszacować błąd predykcji na zbiorze testowym

### Wersja tensorFlow
Podczas zajęć będziemy używać Keras, będącej wysokopoziomowym interfejsem do TensorFlow.
Żeby móc korzystac z Keras, należy go zainstalować. Wykonaj poniższy kod

In [ ]:
# https://keras.io/
#!pip install -q keras
from keras.layers import Dense
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD
import tensorflow as tf

In [ ]:
# Tworzymy model. Wszystko w TF i Keras to tak naprawdę sieć neuronowa.
# Ale... sieć neuronowa to tylko kombinacja liniowa wejść i wag -- to samo co regresja liniowa
model = Sequential([
        Dense(1, activation='linear', input_shape=(1,))
    ])

# Optymalizator wykorzystywany w Keras, to w naszym przypadku Stochastic Gradient Descent
# Spróbuj różne wartości lr (learning rate) i epochs.
# Odpowiedz na pytanie "co się dzieje????""
learning_rate = 0.01

opti=SGD(lr=learning_rate, momentum=0.0)



model.compile(loss='mse', optimizer=opti)
model.fit(x=Tr['sqft_living'], y=Tr['price'], batch_size=1, epochs=10, shuffle=False)

#### Zwizualizuj model analogicznie do przypadku z scikit

In [ ]:
w=model.get_weights()
theta_1 = w[0][0][0]
theta_0 =[1][0]


#### Wykorzystaj funkcję [predict](https://keras.io/models/model/#methods) na zbiorze testowym, żeby wyznaczyć ceny szacowane przez model.

#### Wykorzystj funkcję [mean_squared_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) żeby oszacować błąd predykcji na zbiorze testowym

## Porównaj wyniki dla scikit i Keras da różnych zestawów cech
Pamietaj żeby korzystać z zbioru testowego. Do porównania używaj funkcji `mean_squared_error` z poprzednich ćwiczeń

## Implementacja algorytmu Gradient Descent (dodatkowe punkty)
Poniżej przedstawiony został szkielet dwóch funkcji odpowiedzialnych za realizację algorytmu Gradient descent.
Uzupełnij implementację korzystajac z materiałów z wykładu.
**Pamietaj, że w przypadku rozwiązania macierzowego, musisz dodać do zbioru cech X sztuczną cechę wypełnioną 1-kami, przez którą mnożony będzie wyraz wolny theta_0**

In [ ]:
def cost_function(X, y, theta):
    """
    cost_function(X, y, theta) oblicza koszt J(theta)

    @param X: macierz z elementami uczącymi nxm
    @param y: wartości rzeczywiste ze zbioru danych m,1
    @param theta: wektor współczynników 1,n
    """
    ## number of training examples
    m = len(y)

    J= None ## Twój kod tutaj, zgodnie ze wzorem podanym na stronie lab

    return J


In [ ]:
def gradient_descent(X, y, theta, alpha, iterations):
    """
    Wykonuje algorytm gradient descent na danych podanych jako parametr.


    @param X: macierz z elementami uczącymi nxm
    @param y: wartości rzeczywiste ze zbioru danych m,1
    @param theta: wektor współczynników 1,n
    @param alpha: współczynnik uczenia, skalar
    @param iterations: ilość iteracji po których algorytm ma zakończyć działanie
    """
    cost_history = [0] * iterations

    for iteration in range(iterations):

        h_x = ## Twój kod tutaj, zgodnie ze wzorem podanym na stronie lab/wykładzie
        gradient = ## Twój kod tutaj, zgodnie ze wzorem podanym na stronie lab/wykładzie
        new_theta = ## Twój kod tutaj, zgodnie ze wzorem podanym na stronie lab/wykładzie

        theta = new_theta
        cost = cost_function(X, y, theta)
        cost_history[iteration] = cost


    return theta, cost_history

### Porównanie działania algorytmu powyżej z pakietem scikit i TensorFlow
Dokonaj porównania działania wersji zaimplementowanej przez Ciebei z poprzednimi wersjami.
  1. **Spróbuj różnych atrubutów, ich kombinacji, itp.**
  2. **Postaraj się zbudować jak najlepszy model (3 dodatkowe punkty dla najlepszego modelu w grupie)** Można pracować w grupach, ale punkty dzielą sie wówczas przez ilość osób w grupie
  3. Przetestuj swój model na danych znajdujących się tutaj: [House sales test](https://szymon.bobek.re/kc_house_data_train.csv). **Uwaga** Nie ruszaj tego zbioru aż do samego końca. Finalny zbiór testowy jest używany tylko raz -- aby sprawdzić jakość modelu. Po takim teście nie poprawiamy więcej modelu. Dlaczego?

## Closed form solution
Rozwiąż problem regresji liniowej wykorzystując `closed form solution`
  1. Porównaj wyniki z metodami gradientowymi
  2. Porównaj czas z metodami gradientowymi
  3. Czy trzeba normalizować cechy w przypadku tego podejścia?